
<a id='merging'></a>
{{ header }}

# Merge, join, and concatenate

pandas provides various facilities for easily combining together Series or
DataFrame with various kinds of set logic for the indexes
and relational algebra functionality in the case of join / merge-type
operations.


<a id='merging-concat'></a>

## Concatenating objects

The `concat()` function (in the main pandas namespace) does all of
the heavy lifting of performing concatenation operations along an axis while
performing optional set logic (union or intersection) of the indexes (if any) on
the other axes. Note that I say “if any” because there is only a single possible
axis of concatenation for Series.

Before diving into all of the details of `concat` and what it can do, here is
a simple example:

Like its sibling function on ndarrays, `numpy.concatenate`, `pandas.concat`
takes a list or dict of homogeneously-typed objects and concatenates them with
some configurable handling of “what to do with the other axes”:

In [ ]:
pd.concat(objs, axis=0, join='outer', ignore_index=False, keys=None,
          levels=None, names=None, verify_integrity=False, copy=True)

- `objs` : a sequence or mapping of Series or DataFrame objects. If a
  dict is passed, the sorted keys will be used as the keys argument, unless
  it is passed, in which case the values will be selected (see below). Any None
  objects will be dropped silently unless they are all None in which case a
  ValueError will be raised.  
- `axis` : {0, 1, …}, default 0. The axis to concatenate along.  
- `join` : {‘inner’, ‘outer’}, default ‘outer’. How to handle indexes on
  other axis(es). Outer for union and inner for intersection.  
- `ignore_index` : boolean, default False. If True, do not use the index
  values on the concatenation axis. The resulting axis will be labeled 0, …,
  n - 1. This is useful if you are concatenating objects where the
  concatenation axis does not have meaningful indexing information. Note
  the index values on the other axes are still respected in the join.  
- `keys` : sequence, default None. Construct hierarchical index using the
  passed keys as the outermost level. If multiple levels passed, should
  contain tuples.  
- `levels` : list of sequences, default None. Specific levels (unique values)
  to use for constructing a MultiIndex. Otherwise they will be inferred from the
  keys.  
- `names` : list, default None. Names for the levels in the resulting
  hierarchical index.  
- `verify_integrity` : boolean, default False. Check whether the new
  concatenated axis contains duplicates. This can be very expensive relative
  to the actual data concatenation.  
- `copy` : boolean, default True. If False, do not copy data unnecessarily.  


Without a little bit of context many of these arguments don’t make much sense.
Let’s revisit the above example. Suppose we wanted to associate specific keys
with each of the pieces of the chopped up DataFrame. We can do this using the
`keys` argument:

As you can see (if you’ve read the rest of the documentation), the resulting
object’s index has a [hierarchical index](advanced.ipynb#advanced-hierarchical). This
means that we can now select out each chunk by key:

It’s not a stretch to see how this can be very useful. More detail on this
functionality below.

>**Note**
>
>It is worth noting that `concat()` (and therefore
`append()`) makes a full copy of the data, and that constantly
reusing this function can create a significant performance hit. If you need
to use the operation over several datasets, use a list comprehension.

In [ ]:
frames = [ process_your_file(f) for f in files ]
result = pd.concat(frames)

### Set logic on the other axes

When gluing together multiple DataFrames, you have a choice of how to handle
the other axes (other than the one being concatenated). This can be done in
the following two ways:

- Take the union of them all, `join='outer'`. This is the default
  option as it results in zero information loss.  
- Take the intersection, `join='inner'`.  


Here is an example of each of these methods. First, the default `join='outer'`
behavior:

Changed in version 0.23.0.

The default behavior with `join='outer'` is to sort the other axis
(columns in this case). In a future version of pandas, the default will
be to not sort. We specified `sort=False` to opt in to the new
behavior now.

Here is the same thing with `join='inner'`:

Lastly, suppose we just wanted to reuse the *exact index* from the original
DataFrame:

Similarly, we could index before the concatenation:


<a id='merging-concatenation'></a>

### Concatenating using `append`

A useful shortcut to `concat()` are the `append()`
instance methods on `Series` and `DataFrame`. These methods actually predated
`concat`. They concatenate along `axis=0`, namely the index:

In the case of `DataFrame`, the indexes must be disjoint but the columns do not
need to be:

`append` may take multiple objects to concatenate:

>**Note**
>
>Unlike the `append()` method, which appends to the original list
and returns `None`, `append()`  here **does not** modify
`df1` and returns its copy with `df2` appended.


<a id='merging-ignore-index'></a>

### Ignoring indexes on the concatenation axis

For `DataFrame` objects which don’t have a meaningful index, you may wish
to append them and ignore the fact that they may have overlapping indexes. To
do this, use the `ignore_index` argument:

This is also a valid argument to `DataFrame.append()`:


<a id='merging-mixed-ndims'></a>

### Concatenating with mixed ndims

You can concatenate a mix of `Series` and `DataFrame` objects. The
`Series` will be transformed to `DataFrame` with the column name as
the name of the `Series`.

>**Note**
>
>Since we’re concatenating a `Series` to a `DataFrame`, we could have
achieved the same result with `DataFrame.assign()`. To concatenate an
arbitrary number of pandas objects (`DataFrame` or `Series`), use
`concat`.

If unnamed `Series` are passed they will be numbered consecutively.

Passing `ignore_index=True` will drop all name references.

### More concatenating with group keys

A fairly common use of the `keys` argument is to override the column names
when creating a new `DataFrame` based on existing `Series`.
Notice how the default behaviour consists on letting the resulting `DataFrame`
inherit the parent `Series`’ name, when these existed.

Through the `keys` argument we can override the existing column names.

Let’s consider a variation of the very first example presented:

You can also pass a dict to `concat` in which case the dict keys will be used
for the `keys` argument (unless other keys are specified):

The MultiIndex created has levels that are constructed from the passed keys and
the index of the `DataFrame` pieces:

If you wish to specify other levels (as will occasionally be the case), you can
do so using the `levels` argument:

This is fairly esoteric, but it is actually necessary for implementing things
like GroupBy where the order of a categorical variable is meaningful.


<a id='merging-append-row'></a>

### Appending rows to a DataFrame

While not especially efficient (since a new object must be created), you can
append a single row to a `DataFrame` by passing a `Series` or dict to
`append`, which returns a new `DataFrame` as above.

You should use `ignore_index` with this method to instruct DataFrame to
discard its index. If you wish to preserve the index, you should construct an
appropriately-indexed DataFrame and append or concatenate those objects.

You can also pass a list of dicts or Series:


<a id='merging-join'></a>

## Database-style DataFrame or named Series joining/merging

pandas has full-featured, **high performance** in-memory join operations
idiomatically very similar to relational databases like SQL. These methods
perform significantly better (in some cases well over an order of magnitude
better) than other open source implementations (like `base::merge.data.frame`
in R). The reason for this is careful algorithmic design and the internal layout
of the data in `DataFrame`.

See the [cookbook](cookbook.ipynb#cookbook-merge) for some advanced strategies.

Users who are familiar with SQL but new to pandas might be interested in a
[comparison with SQL](getting_started/comparison/comparison_with_sql.ipynb#compare-with-sql-join).

pandas provides a single function, `merge()`, as the entry point for
all standard database join operations between `DataFrame` or named `Series` objects:

In [ ]:
pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

- `left`: A DataFrame or named Series object.  
- `right`: Another DataFrame or named Series object.  
- `on`: Column or index level names to join on. Must be found in both the left
  and right DataFrame and/or Series objects. If not passed and `left_index` and
  `right_index` are `False`, the intersection of the columns in the
  DataFrames and/or Series will be inferred to be the join keys.  
- `left_on`: Columns or index levels from the left DataFrame or Series to use as
  keys. Can either be column names, index level names, or arrays with length
  equal to the length of the DataFrame or Series.  
- `right_on`: Columns or index levels from the right DataFrame or Series to use as
  keys. Can either be column names, index level names, or arrays with length
  equal to the length of the DataFrame or Series.  
- `left_index`: If `True`, use the index (row labels) from the left
  DataFrame or Series as its join key(s). In the case of a DataFrame or Series with a MultiIndex
  (hierarchical), the number of levels must match the number of join keys
  from the right DataFrame or Series.  
- `right_index`: Same usage as `left_index` for the right DataFrame or Series  
- `how`: One of `'left'`, `'right'`, `'outer'`, `'inner'`. Defaults
  to `inner`. See below for more detailed description of each method.  
- `sort`: Sort the result DataFrame by the join keys in lexicographical
  order. Defaults to `True`, setting to `False` will improve performance
  substantially in many cases.  
- `suffixes`: A tuple of string suffixes to apply to overlapping
  columns. Defaults to `('_x', '_y')`.  
- `copy`: Always copy data (default `True`) from the passed DataFrame or named Series
  objects, even when reindexing is not necessary. Cannot be avoided in many
  cases but may improve performance / memory usage. The cases where copying
  can be avoided are somewhat pathological but this option is provided
  nonetheless.  
- `indicator`: Add a column to the output DataFrame called `_merge`
  with information on the source of each row. `_merge` is Categorical-type
  and takes on a value of `left_only` for observations whose merge key
  only appears in `'left'` DataFrame or Series, `right_only` for observations whose
  merge key only appears in `'right'` DataFrame or Series, and `both` if the
  observation’s merge key is found in both.  
- `validate` : string, default None.
  If specified, checks if merge is of specified type.  
  
  - “one_to_one” or “1:1”: checks if merge keys are unique in both
    left and right datasets.  
  - “one_to_many” or “1:m”: checks if merge keys are unique in left
    dataset.  
  - “many_to_one” or “m:1”: checks if merge keys are unique in right
    dataset.  
  - “many_to_many” or “m:m”: allowed, but does not result in checks.  
  
  New in version 0.21.0.  


>**Note**
>
>Support for specifying index levels as the `on`, `left_on`, and
`right_on` parameters was added in version 0.23.0.
Support for merging named `Series` objects was added in version 0.24.0.

The return type will be the same as `left`. If `left` is a `DataFrame` or named `Series`
and `right` is a subclass of `DataFrame`, the return type will still be `DataFrame`.

`merge` is a function in the pandas namespace, and it is also available as a
`DataFrame` instance method `merge()`, with the calling
`DataFrame` being implicitly considered the left object in the join.

The related `join()` method, uses `merge` internally for the
index-on-index (by default) and column(s)-on-index join. If you are joining on
index only, you may wish to use `DataFrame.join` to save yourself some typing.

### Brief primer on merge methods (relational algebra)

Experienced users of relational databases like SQL will be familiar with the
terminology used to describe join operations between two SQL-table like
structures (`DataFrame` objects). There are several cases to consider which
are very important to understand:

- **one-to-one** joins: for example when joining two `DataFrame` objects on
  their indexes (which must contain unique values).  
- **many-to-one** joins: for example when joining an index (unique) to one or
  more columns in a different `DataFrame`.  
- **many-to-many** joins: joining columns on columns.  


>**Note**
>
>When joining columns on columns (potentially a many-to-many join), any
indexes on the passed `DataFrame` objects **will be discarded**.

It is worth spending some time understanding the result of the **many-to-many**
join case. In SQL / standard relational algebra, if a key combination appears
more than once in both tables, the resulting table will have the **Cartesian
product** of the associated data. Here is a very basic example with one unique
key combination:

Here is a more complicated example with multiple join keys. Only the keys
appearing in `left` and `right` are present (the intersection), since
`how='inner'` by default.

The `how` argument to `merge` specifies how to determine which keys are to
be included in the resulting table. If a key combination **does not appear** in
either the left or right tables, the values in the joined table will be
`NA`. Here is a summary of the `how` options and their SQL equivalent names:

````````````````|Merge method|SQL Join Name|Description|
|:------------------:|:------------------:|:----------------------------------------------------------:|
|left|LEFT OUTER JOIN|Use keys from left frame only|
|right|RIGHT OUTER JOIN|Use keys from right frame only|
|outer|FULL OUTER JOIN|Use union of keys from both frames|
|inner|INNER JOIN|Use intersection of keys from both frames|
Here is another example with duplicate join keys in DataFrames:

Joining / merging on duplicate keys can cause a returned frame that is the multiplication of the row dimensions, which may result in memory overflow. It is the user’ s responsibility to manage duplicate values in keys before joining large DataFrames.


<a id='merging-validation'></a>

### Checking for duplicate keys

New in version 0.21.0.

Users can use the `validate` argument to automatically check whether there
are unexpected duplicates in their merge keys. Key uniqueness is checked before
merge operations and so should protect against memory overflows. Checking key
uniqueness is also a good way to ensure user data structures are as expected.

In the following example, there are duplicate values of `B` in the right
`DataFrame`. As this is not a one-to-one merge – as specified in the
`validate` argument – an exception will be raised.

```ipython
In [53]: result = pd.merge(left, right, on='B', how='outer', validate="one_to_one")
...
MergeError: Merge keys are not unique in right dataset; not a one-to-one merge
```


If the user is aware of the duplicates in the right `DataFrame` but wants to
ensure there are no duplicates in the left DataFrame, one can use the
`validate='one_to_many'` argument instead, which will not raise an exception.


<a id='merging-indicator'></a>

### The merge indicator

`merge()` accepts the argument `indicator`. If `True`, a
Categorical-type column called `_merge` will be added to the output object
that takes on values:

> ````````````|Observation Origin|_merge value|
|:---------------------------------:|:--------------:|
|Merge key only in 'left' frame|left_only|
|Merge key only in 'right' frame|right_only|
|Merge key in both frames|both|

The `indicator` argument will also accept string arguments, in which case the indicator function will use the value of the passed string as the name for the indicator column.


<a id='merging-dtypes'></a>

### Merge dtypes

New in version 0.19.0.

Merging will preserve the dtype of the join keys.

We are able to preserve the join keys:

Of course if you have missing values that are introduced, then the
resulting dtype will be upcast.

New in version 0.20.0.

Merging will preserve `category` dtypes of the mergands. See also the section on [categoricals](categorical.ipynb#categorical-merge).

The left frame.

The right frame.

The merged result:

>**Note**
>
>The category dtypes must be *exactly* the same, meaning the same categories and the ordered attribute.
Otherwise the result will coerce to `object` dtype.

>**Note**
>
>Merging on `category` dtypes that are the same can be quite performant compared to `object` dtype merging.


<a id='merging-join-index'></a>

### Joining on index

`DataFrame.join()` is a convenient method for combining the columns of two
potentially differently-indexed `DataFrames` into a single result
`DataFrame`. Here is a very basic example:

The same as above, but with `how='inner'`.

The data alignment here is on the indexes (row labels). This same behavior can
be achieved using `merge` plus additional arguments instructing it to use the
indexes:

### Joining key columns on an index

`join()` takes an optional `on` argument which may be a column
or multiple column names, which specifies that the passed `DataFrame` is to be
aligned on that column in the `DataFrame`. These two function calls are
completely equivalent:

In [ ]:
left.join(right, on=key_or_keys)
pd.merge(left, right, left_on=key_or_keys, right_index=True,
      how='left', sort=False)

Obviously you can choose whichever form you find more convenient. For
many-to-one joins (where one of the `DataFrame`’s is already indexed by the
join key), using `join` may be more convenient. Here is a simple example:


<a id='merging-multikey-join'></a>
To join on multiple keys, the passed DataFrame must have a `MultiIndex`:

Now this can be joined by passing the two key column names:


<a id='merging-df-inner-join'></a>
The default for `DataFrame.join` is to perform a left join (essentially a
“VLOOKUP” operation, for Excel users), which uses only the keys found in the
calling DataFrame. Other join types, for example inner join, can be just as
easily performed:

As you can see, this drops any rows where there was no match.


<a id='merging-join-on-mi'></a>

### Joining a single Index to a MultiIndex

You can join a singly-indexed `DataFrame` with a level of a MultiIndexed `DataFrame`.
The level will match on the name of the index of the singly-indexed frame against
a level name of the MultiIndexed frame.

This is equivalent but less verbose and more memory efficient / faster than this.


<a id='merging-join-with-two-multi-indexes'></a>

### Joining with two MultiIndexes

This is supported in a limited way, provided that the index for the right
argument is completely used in the join, and is a subset of the indices in
the left argument, as in this example:

If that condition is not satisfied, a join with two multi-indexes can be
done using the following code.


<a id='merging-merge-on-columns-and-levels'></a>

### Merging on a combination of columns and index levels

New in version 0.23.

Strings passed as the `on`, `left_on`, and `right_on` parameters
may refer to either column names or index level names.  This enables merging
`DataFrame` instances on a combination of index levels and columns without
resetting indexes.

>**Note**
>
>When DataFrames are merged on a string that matches an index level in both
frames, the index level is preserved as an index level in the resulting
DataFrame.

>**Note**
>
>When DataFrames are merged using only some of the levels of a MultiIndex,
the extra levels will be dropped from the resulting merge. In order to
preserve those levels, use `reset_index` on those level names to move
those levels to columns prior to doing the merge.

>**Note**
>
>If a string matches both a column name and an index level name, then a
warning is issued and the column takes precedence. This will result in an
ambiguity error in a future version.

### Overlapping value columns

The merge `suffixes` argument takes a tuple of list of strings to append to
overlapping column names in the input `DataFrame`s to disambiguate the result
columns:

`DataFrame.join()` has `lsuffix` and `rsuffix` arguments which behave
similarly.


<a id='merging-multiple-join'></a>

### Joining multiple DataFrames

A list or tuple of `DataFrames` can also be passed to `join()`
to join them together on their indexes.


<a id='merging-combine-first-update'></a>

### Merging together values within Series or DataFrame columns

Another fairly common situation is to have two like-indexed (or similarly
indexed) `Series` or `DataFrame` objects and wanting to “patch” values in
one object from values for matching indices in the other. Here is an example:

For this, use the `combine_first()` method:

Note that this method only takes values from the right `DataFrame` if they are
missing in the left `DataFrame`. A related method, `update()`,
alters non-NA values in place:


<a id='merging-time-series'></a>

## Timeseries friendly merging


<a id='merging-merge-ordered'></a>

### Merging ordered data

A `merge_ordered()` function allows combining time series and other
ordered data. In particular it has an optional `fill_method` keyword to
fill/interpolate missing data:


<a id='merging-merge-asof'></a>

### Merging asof

New in version 0.19.0.

A `merge_asof()` is similar to an ordered left-join except that we match on
nearest key rather than equal keys. For each row in the `left` `DataFrame`,
we select the last row in the `right` `DataFrame` whose `on` key is less
than the left’s key. Both DataFrames must be sorted by the key.

Optionally an asof merge can perform a group-wise merge. This matches the
`by` key equally, in addition to the nearest match on the `on` key.

For example; we might have `trades` and `quotes` and we want to `asof`
merge them.

By default we are taking the asof of the quotes.

We only asof within `2ms` between the quote time and the trade time.

We only asof within `10ms` between the quote time and the trade time and we
exclude exact matches on time. Note that though we exclude the exact matches
(of the quotes), prior quotes **do** propagate to that point in time.